In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib
import numpy as np
import math
from matplotlib.pyplot import figure
import seaborn as sns
import string
from matplotlib import pyplot as plt
import plotly.io as pio
from matplotlib_venn import venn3
from matplotlib_venn import venn2, venn2_circles
# import venn
from matplotlib import cm
from matplotlib.colors import ListedColormap
from venn import venn
import plotly.express as px

In [ ]:
def list_to_string(lst, symbol):
    return (symbol.join([str(elem) for elem in lst]))

In [ ]:
cd = os.getcwd()
cd

In [ ]:
os.chdir(cd)
os.chdir('results')

# Read Identified

In [ ]:
cys_df = pd.read_csv('cysteineomedb_id_rxt.csv')

In [ ]:
cys_df['cysteineid'] = cys_df['cysteineid'].replace(' ', '')

In [ ]:
cys_df['proteinid'] = cys_df['proteinid'].replace(' ', '')

In [ ]:
c_df = cys_df[cys_df['level'] == 'cysteine']

In [ ]:
p_df = cys_df[cys_df['level'] == 'protein']

In [ ]:
c_df.shape, p_df.shape

# Read Ligandable Datasets

In [ ]:
os.getcwd()

In [ ]:
discovery_df = pd.read_csv('backus_ligandable_dataset.csv')

In [ ]:
suzuki_df = pd.read_csv('cao_ligandable_dataset.csv')

In [ ]:
suzuki_df['cysteineid'] = suzuki_df['cysteineid'].replace(' ', '')

In [ ]:
suzuki_df['proteinid'] = suzuki_df['proteinid'].replace(' ', '')

In [ ]:
tcell_df = pd.read_csv('vinogradova_ligandable_dataset.csv')

In [ ]:
sp3_df = pd.read_csv('yan_ligandable_dataset.csv')

In [ ]:
slcabpp_df = pd.read_csv('kuljanin_ligandable_dataset.csv')

In [ ]:
dia_df = pd.read_csv('yang_ligandable_dataset.csv')

In [ ]:
def get_lig_ids(dfs, cysteine):
    c_ids = []
    
    for i in range(len(dfs)):
        current_df = dfs[i]
        current_df['proteinid'] = current_df['cysteineid'].map(lambda x: str(x).replace(' ', '').split('_')[0])
        
        lig_current_df = current_df[current_df['ligandable'] == 'yes']
        print(lig_current_df.shape)
        
        if cysteine == True:
            c_ids += list(lig_current_df['cysteineid'].unique())
        else:
            c_ids += list(lig_current_df['proteinid'].unique())
        
    unique_c_ids = list(set(c_ids))
    return unique_c_ids

In [ ]:
unique_c_ids = get_lig_ids([discovery_df,
                           suzuki_df,
                           tcell_df,
                           sp3_df,
                           slcabpp_df,
                           dia_df], True)

In [ ]:
len(unique_c_ids)

In [ ]:
unique_p_ids = get_lig_ids([discovery_df,
                           suzuki_df,
                           tcell_df,
                           sp3_df,
                           slcabpp_df,
                           dia_df], False)

In [ ]:
len(unique_p_ids)

In [ ]:
c_df_subset = c_df.copy()

In [ ]:
c_df_subset.shape

In [ ]:
new_discovery_df = discovery_df.rename(columns = {'ligandable':'backus_cravatt_ligandable'})
c_df_subset = pd.merge(c_df_subset, new_discovery_df[['cysteineid', 'backus_cravatt_ligandable']], on = 'cysteineid', how = 'left')

In [ ]:
new_suzuki_df = suzuki_df.rename(columns = {'ligandable':'cao_backus_ligandable'})
c_df_subset = pd.merge(c_df_subset, new_suzuki_df[['cysteineid', 'cao_backus_ligandable']], on = 'cysteineid', how = 'left')

In [ ]:
new_tcell_df = tcell_df.rename(columns = {'ligandable':'vinogradova_cravatt_ligandable'})
c_df_subset = pd.merge(c_df_subset, new_tcell_df[['cysteineid', 'vinogradova_cravatt_ligandable']], on = 'cysteineid', how = 'left')

In [ ]:
new_sp3_df = sp3_df.rename(columns = {'ligandable':'yan_backus_ligandable'})
c_df_subset = pd.merge(c_df_subset, new_sp3_df[['cysteineid', 'yan_backus_ligandable']], on = 'cysteineid', how = 'left')

In [ ]:
new_slcabpp_df = slcabpp_df.rename(columns = {'ligandable':'kuljanin_gygi_ligandable'})
c_df_subset = pd.merge(c_df_subset, new_slcabpp_df[['cysteineid', 'kuljanin_gygi_ligandable']], on = 'cysteineid', how = 'left')

In [ ]:
new_dia_df = slcabpp_df.rename(columns = {'ligandable':'yang_wang_ligandable'})
c_df_subset = pd.merge(c_df_subset, new_dia_df[['cysteineid', 'yang_wang_ligandable']], on = 'cysteineid', how = 'left')

In [ ]:
test = pd.merge(c_df_subset, new_suzuki_df, on = 'cysteineid', how = 'outer')

In [ ]:
def get_ligandable_datasets(df):
    datasets = []
    
    for index, row in df.iterrows():
        current_datasets = ''
        if row['backus_cravatt_ligandable'] == 'yes':
            current_datasets += 'backus_cravatt' + ';'
        if row['cao_backus_ligandable'] == 'yes':
            current_datasets += 'cao_backus' + ';'
        if row['vinogradova_cravatt_ligandable'] == 'yes':
            current_datasets += 'vinogradova_cravatt' + ';'
        if row['yan_backus_ligandable'] == 'yes':
            current_datasets += 'yan_backus' + ';'
        if row['kuljanin_gygi_ligandable'] == 'yes':
            current_datasets += 'kuljanin_gygi' + ';'
        if row['yang_wang_ligandable'] == 'yes':
            current_datasets += 'yang_wang' + ';'
            
        datasets.append(current_datasets[:-1])
    return datasets

In [ ]:
datasets = get_ligandable_datasets(c_df_subset)

In [ ]:
c_df['ligandable_datasets'] = datasets

In [ ]:
c_df['ligandable'] = np.where(c_df['cysteineid'].isin(unique_c_ids), 'yes', None)

In [ ]:
c_df['ligandable'].value_counts()

In [ ]:
subset_p_df = p_df.copy()

In [ ]:
def get_ligandable_pro_ids(df, subset_p_df, dataset):
    lig_df = df[df[dataset] == 'yes']
    p_lig_ids = list(lig_df['proteinid'].unique())
    
    subset_p_df[dataset] = np.where(subset_p_df['proteinid'].isin(p_lig_ids), 'yes', None)
    return subset_p_df

In [ ]:
subset_p_df = get_ligandable_pro_ids(new_discovery_df, subset_p_df, 'backus_cravatt_ligandable')

In [ ]:
subset_p_df = get_ligandable_pro_ids(new_suzuki_df, subset_p_df, 'cao_backus_ligandable')

In [ ]:
subset_p_df = get_ligandable_pro_ids(new_tcell_df, subset_p_df, 'vinogradova_cravatt_ligandable')

In [ ]:
subset_p_df = get_ligandable_pro_ids(new_sp3_df, subset_p_df, 'yan_backus_ligandable')

In [ ]:
subset_p_df = get_ligandable_pro_ids(new_slcabpp_df, subset_p_df, 'kuljanin_gygi_ligandable')

In [ ]:
subset_p_df = get_ligandable_pro_ids(new_dia_df, subset_p_df, 'yang_wang_ligandable')

In [ ]:
p_df['ligandable_datasets'] = p_datasets

In [ ]:
p_df['ligandable'] = np.where(p_df['proteinid'].isin(unique_p_ids), 'yes', None)

In [ ]:
p_df['ligandable'].value_counts()

In [ ]:
id_df = pd.concat([c_df, p_df])

In [ ]:
id_df.shape

In [ ]:
id_df.to_csv('cysteineomedb_id_rxt_lig.csv', index = False)

# Concatonated Ligandable Data

In [ ]:
cysdb_datasetid_df = pd.read_csv('cysteineomedb_datasetid_rxt.csv')

In [ ]:
cysdb_datasetid_df 

In [ ]:
c_cysdb_datasetid_df = cysdb_datasetid_df[cysdb_datasetid_df['level'] == 'cysteine']

In [ ]:
p_cysdb_datasetid_df = cysdb_datasetid_df[cysdb_datasetid_df['level'] == 'protein']

In [ ]:
def get_new_concat_df(df, cysdb_df, dataset, lig_name, name):
    current_df = cysdb_df[cysdb_df['dataset'] == dataset]
    cys_df = current_df[current_df['level'] == 'cysteine']
    pro_df = current_df[['proteinid']]
    pro_df = pro_df.drop_duplicates()
    
    ligandable = df[df[lig_name] == 'yes']
    ligandable_cys_ids = list(ligandable['cysteineid'].unique())
    ligandable_pro_ids = list(ligandable['proteinid'].unique())
    
    cys_df['ligandable'] = np.where(cys_df['cysteineid'].isin(ligandable_cys_ids), 'yes', None)
    cys_df['ligandable_datasets'] = np.where(cys_df['cysteineid'].isin(ligandable_cys_ids), name, None)
    cys_df = cys_df[['cysteineid', 'ligandable', 'ligandable_datasets']]
    cys_df['dataset'] = name
    
    pro_df['ligandable'] = np.where(pro_df['proteinid'].isin(ligandable_pro_ids), 'yes', None)
    pro_df['ligandable_datasets'] = np.where(pro_df['proteinid'].isin(ligandable_pro_ids), name, None)
    pro_df['dataset'] = name
    
    return cys_df, pro_df

In [ ]:
cys_discovery_df, pro_discovery_df = get_new_concat_df(c_df_subset, c_cysdb_datasetid_df, 'backus_cravatt', 'backus_cravatt_ligandable', 'backus_cravatt')

In [ ]:
cys_suzuki_df, pro_suzuki_df = get_new_concat_df(c_df_subset, c_cysdb_datasetid_df, 'cao_backus', 'cao_backus_ligandable', 'cao_backus')

In [ ]:
cys_sp3_df, pro_sp3_df = get_new_concat_df(c_df_subset, c_cysdb_datasetid_df, 'yan_backus', 'yan_backus_ligandable', 'yan_backus')

In [ ]:
cys_tcell_df, pro_tcell_df = get_new_concat_df(c_df_subset, c_cysdb_datasetid_df, 'vinogradova_cravatt', 'vinogradova_cravatt_ligandable', 'vinogradova_cravatt')

In [ ]:
cys_slcabpp_df, pro_slcabpp_df = get_new_concat_df(c_df_subset, c_cysdb_datasetid_df, 'kuljanin_gygi', 'kuljanin_gygi_ligandable', 'kuljanin_gygi')

In [ ]:
cys_dia_df, pro_dia_df = get_new_concat_df(c_df_subset, c_cysdb_datasetid_df, 'yang_wang', 'yang_wang_ligandable', 'yang_wang')

In [ ]:
new_c_df = pd.concat([cys_discovery_df,
                     cys_suzuki_df,
                     cys_sp3_df,
                     cys_tcell_df,
                     cys_slcabpp_df,
                     cys_dia_df])

In [ ]:
new_p_df = pd.concat([pro_discovery_df,
                     pro_suzuki_df,
                     pro_sp3_df,
                     pro_tcell_df,
                     pro_slcabpp_df,
                     pro_dia_df])

In [ ]:
new_c_datasetid_df = pd.merge(c_cysdb_datasetid_df, new_c_df, left_on = ['cysteineid', 'dataset'], right_on = ['cysteineid', 'dataset'], how = 'left')

In [ ]:
new_p_datasetid_df = pd.merge(p_cysdb_datasetid_df, new_p_df, left_on = ['proteinid', 'dataset'], right_on = ['proteinid', 'dataset'], how =  'left')

In [ ]:
new_datasetid_df = pd.concat([new_c_datasetid_df, new_p_datasetid_df])

In [ ]:
new_datasetid_df.to_csv('cysteineomedb_datasetid_rxt_lig.csv', index = False)

In [ ]:
new_datasetid_df['ligandable'].value_counts()